In [1]:
import os
import nltk  # Импорт библиотеки Natural Language Toolkit (NLTK)
import csv  # Импорт модуля для работы с файлами CSV

from nltk.corpus import stopwords  # Импорт списка стоп-слов из NLTK
from nltk.stem import SnowballStemmer  # Импорт стеммера из NLTK
from nltk.tokenize import word_tokenize  # Функция для токенизации текста
from nltk.translate.bleu_score import corpus_bleu  # Функция для вычисления BLEU
from nltk.translate.meteor_score import meteor_score  # Функция для вычисления METEOR
from rouge import Rouge  # Класс для вычисления метрики ROUGE




In [2]:
# Создание экземпляра стеммера
stemmer = SnowballStemmer("russian")

# Получение списка стоп-слов на русском языке
stop_words = set(stopwords.words('russian'))

# Функция предобработки текста
def preprocess_text(text):
    # Токенизация текста
    tokens = word_tokenize(text)
    
    # Удаление стоп-слов
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Стемминг токенов
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    
    # Ограничение количества токенов до 512
    limited_tokens = stemmed_tokens[:512]
    
    # Склеивание токенов обратно в строку
    preprocessed_text = ' '.join(limited_tokens)
    
    return preprocessed_text

In [3]:
# Функция для загрузки текста из файла с предобработкой
def load_text_preprocessed(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        # Предобработка загруженного текста
        preprocessed_text = preprocess_text(text)
    return preprocessed_text

# Загрузка предобработанного эталонного текста
reference_text = load_text_preprocessed('reference.txt')

# Загрузка предобработанного переведенного текста
translated_text = load_text_preprocessed('translated.txt')

# Токенизация текстов
reference_tokens = word_tokenize(reference_text.lower())
translated_tokens = word_tokenize(translated_text.lower())

In [4]:
# Вычисление BLEU-метрики
bleu_score = corpus_bleu([[reference_tokens]], [translated_tokens])

# Создание экземпляра класса Rouge
rouge = Rouge()

# Вычисление метрики ROUGE
scores = rouge.get_scores(translated_text, reference_text)

# Оценка качества перевода с помощью метрики METEOR
score = meteor_score([reference_tokens], translated_tokens)

# Вывод результатов
print("BLEU score:", bleu_score)
print("ROUGE scores:", scores)
print("METEOR score:", score)


BLEU score: 1.821831989445342e-231
ROUGE scores: [{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]
METEOR score: 0.5


c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\PC\AppData\Local\Programs\Python\Python38\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it c

In [5]:
# Вычисление среднего арифметического из всех метрик
average = (bleu_score + sum(rouge['rouge-1']['f'] for rouge in scores) + sum(rouge['rouge-2']['f'] for rouge in scores) + sum(rouge['rouge-l']['f'] for rouge in scores) + score) / 5

# Вывод среднего арифметического из всех метрик
print("Среднее арифметическое из всех метрик:", average)

Среднее арифметическое из всех метрик: 0.499999998


In [6]:
# Вычисление суммы всех метрик
k = bleu_score + sum(rouge['rouge-1']['f'] for rouge in scores) + score 

# Уникальные веса для каждой метрики оценки качества перевода художественных текстов
w_bleu = 0.1
w_rouge_1 = 0.5
w_score = 0.4

# Вычисление метрики оценки качества перевода художественных текстов с уникальными весами
art_met = w_bleu * bleu_score + w_rouge_1 * sum(rouge['rouge-1']['f'] for rouge in scores) + w_score * score 

# Результаты оценки качества перевода художественного текста
print("Оценка качества перевода художественного текста:", art_met)

Оценка качества перевода художественного текста: 0.6999999975


In [7]:
def save_metrics_to_csv(metrics, csv_file):
    # Проверяем, существует ли файл CSV
    file_exists = os.path.exists(csv_file)
    
    # Открываем файл CSV в режиме добавления данных
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=metrics.keys())
        
        # Если файл не существует, записываем заголовки полей
        if not file_exists:
            writer.writeheader()
        
        # Записываем значения метрик в файл CSV
        writer.writerow(metrics)

# Пример использования:
metrics = {
    'BLEU score': bleu_score,
    'ROUGE scores': scores,
    'METEOR score': score,
    'Среднее арифметическое из всех метрик': average,
    'Оценка качества перевода': art_met
}

# Путь к файлу CSV
csv_file = 'En-Ru metrics.csv'

# Сохраняем метрики в файл CSV
save_metrics_to_csv(metrics, csv_file)


In [8]:
def load_metrics_from_csv(csv_file):
    metrics = []
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            metrics.append(row)
    return metrics

# Пример использования:
csv_file = 'En-Ru metrics.csv'

# Загружаем данные из файла CSV
metrics = load_metrics_from_csv(csv_file)

# Отображаем значения метрик
for entry in metrics:
    print(entry)


{'BLEU score': '1.821831989445342e-231', 'ROUGE scores': "[{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}]", 'METEOR score': '0.5', 'Среднее арифметическое из всех метрик': '0.499999998', 'Оценка качества перевода': '0.6999999975'}


In [9]:
def load_metrics_from_csv(csv_file):
    metrics = []
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            metrics.append(row)
    return metrics

def calculate_average(metrics):
    bleu_scores = []
    meteor_scores = []
    rouge_1_scores = []
    rouge_2_scores = []
    rouge_l_scores = []
    average_scores = []
    art_met_scores = []  # Массив для хранения значений art_met
    
    for entry in metrics:
        bleu_scores.append(float(entry['BLEU score']))
        meteor_scores.append(float(entry['METEOR score']))
        
        # ROUGE scores в формате строки, преобразуем в словарь
        rouge_scores = eval(entry['ROUGE scores'])
        
        rouge_1_scores.append(rouge_scores[0]['rouge-1']['f'])
        rouge_2_scores.append(rouge_scores[0]['rouge-2']['f'])
        rouge_l_scores.append(rouge_scores[0]['rouge-l']['f'])
        
        average_scores.append(float(entry['Среднее арифметическое из всех метрик']))
        art_met_scores.append(float(entry['Оценка качества перевода']))  # Добавляем значение art_met из CSV
    
    # Вычисляем среднее арифметическое для каждой метрики
    average_bleu = sum(bleu_scores) / len(bleu_scores)
    average_meteor = sum(meteor_scores) / len(meteor_scores)
    average_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores)
    average_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)
    average_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
    average = sum(average_scores) / len(average_scores)
    average_art_met = sum(art_met_scores) / len(art_met_scores)  # Среднее для art_met
    
    return average_bleu, average_meteor, average_rouge_1, average_rouge_2, average_rouge_l, average, average_art_met

# Пример использования:
csv_file = 'En-Ru metrics.csv'

# Загружаем данные из файла CSV
metrics = load_metrics_from_csv(csv_file)

# Вычисляем среднее арифметическое
average_bleu, average_meteor, average_rouge_1, average_rouge_2, average_rouge_l, average, average_art_met = calculate_average(metrics)

# Выводим среднее арифметическое
print("Среднее арифметическое для BLEU score:", average_bleu)
print("Среднее арифметическое для METEOR score:", average_meteor)
print("Среднее арифметическое для ROUGE-1:", average_rouge_1)
print("Среднее арифметическое для ROUGE-2:", average_rouge_2)
print("Среднее арифметическое для ROUGE-L:", average_rouge_l)
print("Среднее арифметическое из всех метрик:", average)
print("Оценка качества перевода:", average_art_met)  


Среднее арифметическое для BLEU score: 1.821831989445342e-231
Среднее арифметическое для METEOR score: 0.5
Среднее арифметическое для ROUGE-1: 0.999999995
Среднее арифметическое для ROUGE-2: 0.0
Среднее арифметическое для ROUGE-L: 0.999999995
Среднее арифметическое из всех метрик: 0.499999998
Оценка качества перевода: 0.6999999975


In [14]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score

def load_file(entry):
    filepath = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if filepath:
        entry.delete(0, tk.END)
        entry.insert(0, filepath)

def process_files():
    ref_path = entry_ref.get()
    trans_path = entry_trans.get()

    if not ref_path or not trans_path:
        messagebox.showerror("Ошибка", "Необходимо загрузить оба файла")
        return

    try:
        # Загрузка и обработка текстов
        reference_text = load_text_preprocessed(ref_path)
        translated_text = load_text_preprocessed(trans_path)

        # Подсчет метрик и вывод результатов
        calculate_and_display_metrics(reference_text, translated_text)
    except Exception as e:
        messagebox.showerror("Ошибка при обработке файлов", str(e))

def calculate_and_display_metrics(ref_text, trans_text):
    # Токенизация текстов
    reference_tokens = word_tokenize(ref_text.lower())
    translated_tokens = word_tokenize(trans_text.lower())

    # Вычисление BLEU-метрики
    bleu_score = corpus_bleu([[reference_tokens]], [translated_tokens])

    # Создание экземпляра класса Rouge
    rouge = Rouge()

    # Вычисление метрики ROUGE
    scores = rouge.get_scores(trans_text, ref_text)

    # Оценка качества перевода с помощью метрики METEOR
    meteor_score_value = meteor_score([reference_tokens], translated_tokens)

    # Выбор весов для каждой метрики в зависимости от типа текста
    if metric_type.get() == 'art':
        w_bleu, w_rouge_1, w_score = 0.10, 0.50, 0.40
    elif metric_type.get() == 'tech':
        w_bleu, w_rouge_1, w_score = 0.20, 0.40, 0.40
    elif metric_type.get() == 'doc':
        w_bleu, w_rouge_1, w_score = 0.20, 0.45, 0.35

    # Вычисление новой художественной метрики с весами
    art_met = w_bleu * bleu_score + w_rouge_1 * scores[0]['rouge-1']['f'] + w_score * meteor_score_value

    # Вывод результатов
    result_text = f"BLEU score: {bleu_score:.3f}\n"
    result_text += f"ROUGE scores: {scores[0]['rouge-1']['f']:.3f}, {scores[0]['rouge-2']['f']:.3f}, {scores[0]['rouge-l']['f']:.3f}\n"
    result_text += f"METEOR score: {meteor_score_value:.3f}\n"
    result_text += f"Оценка качества перевода (Выбранная метрика): {art_met:.3f}\n"
    
    text_widget.delete('1.0', tk.END)
    text_widget.insert(tk.END, result_text)

def load_text_preprocessed(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Настройка основного окна
root = tk.Tk()
root.title("MetriX")

# Настройка фрейма для загрузки файлов
frame = tk.Frame(root)
frame.pack(pady=20)

entry_ref = tk.Entry(frame, width=50)
entry_ref.pack(side=tk.LEFT, padx=(0, 10))
button_ref = tk.Button(frame, text="Загрузить файл профессионального перевода", command=lambda: load_file(entry_ref))
button_ref.pack(side=tk.LEFT)

frame_trans = tk.Frame(root)
frame_trans.pack(pady=10)

entry_trans = tk.Entry(frame_trans, width=50)
entry_trans.pack(side=tk.LEFT, padx=(0, 10))
button_trans = tk.Button(frame_trans, text="Загрузить файл системы машинного перевода", command=lambda: load_file(entry_trans))
button_trans.pack(side=tk.LEFT)

# Добавляем выбор метрики
metric_type = tk.StringVar(value='art')
frame_metric = tk.Frame(root)
frame_metric.pack(pady=10)
tk.Label(frame_metric, text="Выберите тип текста:").pack(side=tk.LEFT, padx=(0, 10))
tk.Radiobutton(frame_metric, text="Художественный текст", variable=metric_type, value='art').pack(side=tk.LEFT)
tk.Radiobutton(frame_metric, text="Технический текст", variable=metric_type, value='tech').pack(side=tk.LEFT)
tk.Radiobutton(frame_metric, text="Документационный текст", variable=metric_type, value='doc').pack(side=tk.LEFT)

process_button = tk.Button(root, text="Обработать и показать метрики", command=process_files)
process_button.pack(pady=20)

# Текстовый виджет для вывода результатов
text_widget = tk.Text(root, height=15, width=80)
text_widget.pack(pady=10)

root.mainloop()
